<a href="https://colab.research.google.com/github/HadasRavikovitch/Final-Project---GPU/blob/main/GPU_simplify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numba import cuda, jit
import numpy as np

# just for simple version
@cuda.jit
def gaussian(d_a, d_b, d_pulse):
    d_pulse = d_a * d_b

In [ ]:
from numba import cuda
import math
import numpy as np

# just for simple version
@cuda.jit
def llg(d_c, d_e, d_llg):
    d_result = d_c - d_e

In [ ]:
# just for simple version

@cuda.jit
def simulation(d_a, d_b, d_c, d_e, d_pulse, d_llg, arr_res):
    for i in range(5):
      gaussian(d_a, d_b, d_pulse)
      llgi(d_c, d_e, d_llg)
      arr_res[i] = d_pulse * d_llg
      print(arr_res[i])

In [ ]:
from numba import cuda, jit, float32
import math
import numpy as np
import cupy as cp

a = np.arange((5),dtype=np.float32)
b = np.ones((5,), dtype=np.float32)
c = np.zeros((5,), dtype=np.float32)
# Fix: Convert e to np.float32 before assigning it to d_e
e = np.arange(2,7, dtype=np.float32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)
d_e = cuda.to_device(e)
d_pulse = cuda.device_array((5,), dtype=np.float32)
d_llg = cuda.device_array((5,), dtype=np.float32)
d_out = cuda.device_array((5,), dtype=np.float32)

# just for simple version

@cuda.jit
def gaussian(d_a, d_b, d_gues):
    """Calculate element-wise product of d_a and d_b."""
    idx = cuda.grid(1)
    if idx < d_a.shape[0]:
      d_gues = d_a[idx] * d_b[idx]

@cuda.jit
def llg(d_c, d_e, d_llg):
  """Calculate element-wise difference of d_c and d_e."""
  idx = cuda.grid(1)
  if idx < d_c.shape[0]:
    d_llg[idx] = d_c[idx] - d_e[idx]


@cuda.jit
def simulation(d_a, d_b, d_c, d_e, d_pulse, d_llg, arr_res):
      gaussian(d_a, d_b, d_pulse)
      llg(d_c, d_e, d_llg)
      idx = cuda.grid(1)
      if idx < arr_res.shape[0]:
        arr_res[idx] = d_pulse[idx] + d_llg[idx]

simulation[1, 5](d_a, d_b, d_c, d_e, d_pulse, d_llg, d_out)
print(d_out.copy_to_host())

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[-2. -3. -4. -5. -6.]


In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def cross(array1, array2):
  cross_result = cuda.device_array_like(array1)
  idx = cuda.grid(1)
  if idx < array1.shape[0]:
    cross_result[idx][0] = array1[idx][1] * array2[idx][2] - array1[idx][2] * array2[idx][1]
    cross_result[idx][1] = array1[idx][2] * array2[idx][0] - array1[idx][0] * array2[idx][2]
    cross_result[idx][2] = array1[idx][0] * array2[idx][1] - array1[idx][1] * array2[idx][0]

  return cross_result

    M0 = np.linalg.norm(M, axis=1, keepdims=True)
    gma_LL=gma/((1+alpha**2))
    LL_lamda=gma*alpha/(1+alpha**2)
    
   # Compute LLG terms
    an = -gma_LL * miu * np.cross(M, H, axis=1) - (LL_lamda * miu / M0) * np.cross(M, np.cross(M, H, axis=1), axis=1)
    
    bn = -gma_LL * miu * np.cross(M + (dt / 2) * an, H, axis=1) - (LL_lamda * miu / M0) * np.cross(M + (dt / 2) * an, np.cross(M + (dt / 2) * an, H, axis=1), axis=1)
    
    cn = -gma_LL * miu * np.cross(M + (dt / 2) * bn, H, axis=1) - (LL_lamda * miu / M0) * np.cross(M + (dt / 2) * bn, np.cross(M + (dt / 2) * bn, H, axis=1), axis=1)
    
    dn = -gma_LL * miu * np.cross(M + dt * cn, H, axis=1) - (LL_lamda * miu / M0) * np.cross(M + dt * cn, np.cross(M + dt * cn, H, axis=1), axis=1)
    
    new_M = M + (dt/6)*(an+2*bn+2*cn+dn)
    return new_M

In [ ]:
import numpy as np
from numba import cuda

x = np.array([[1,2,3], [4,5,6]])
y = np.array([[4,5,6], [1,7,3]])

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_cross = cuda.device_array_like(d_x)

res1= np.cross(x, y, axis=1)
print(res1)


[[ -3   6  -3]
 [-27  -6  23]]


In [ ]:
%timeit np.cross(x, y)

29.8 µs ± 5.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
@cuda.jit
def cross(array1, array2, cross_result): # Add cross_result as an output argument
  idx = cuda.grid(1)
  if idx < array1.shape[0]:
    cross_result[idx][0] = array1[idx][1] * array2[idx][2] - array1[idx][2] * array2[idx][1]
    cross_result[idx][1] = array1[idx][2] * array2[idx][0] - array1[idx][0] * array2[idx][2]
    cross_result[idx][2] = array1[idx][0] * array2[idx][1] - array1[idx][1] * array2[idx][0]

In [ ]:
%timeit cross[4, 16](d_x, d_y, d_cross)

The slowest run took 4.00 times longer than the fastest. This could mean that an intermediate result is being cached.
98.1 µs ± 64.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [ ]:
big_arr1 = np.random.rand(100000, 3)
big_arr2 = np.random.rand(100000, 3)

d_big_arr1 = cuda.to_device(big_arr1)
d_big_arr2 = cuda.to_device(big_arr2)
d_big_res = cuda.device_array_like(d_big_arr1)

#d_big_arr = cross[4,16](d_big_arr1, d_big_arr2, d_big_res)

In [ ]:
%timeit cross[4,16](d_big_arr1, d_big_arr2, d_big_res)

The slowest run took 4.11 times longer than the fastest. This could mean that an intermediate result is being cached.
95.5 µs ± 66.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit np.cross(big_arr1, big_arr2)

5.44 ms ± 64.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# an example
import numpy as np
from numba import cuda

# CPU version
@jit(nopython=True)
def matriz_mult_cpu(A, B):
    result = np.zeros_like(A)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            for k in range(B.shape[1]):
                result[i][j] += A[i][k] * B[k][j]
    return result

# GPU version
@cuda.jit
def matriz_mult_gpu(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

# CPU and GPU times
times_cpu = []
times_gpu = []

for N in sizes:
    # Create arrays of size NxN
    A = np.random.rand(N, N).astype(np.float32)
    B = np.random.rand(N, N).astype(np.float32)
    C = np.zeros_like(A)

    # Calculates CPU time
    start_time_cpu = time.time()
    C_cpu = matriz_mult_cpu(A, B)
    end_time_cpu = time.time()

    # Configure the grid and block
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(A.shape[0] / threadsperblock[1]))
    blockspergrid_y = int(np.ceil(B.shape[1] / threadsperblock[0]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    # Calculates time for GPU
    start_time_gpu = time.time()
    matriz_mult_gpu[blockspergrid, threadsperblock](A, B, C)
    end_time_gpu = time.time()

    times_cpu.append(end_time_cpu - start_time_cpu)
    times_gpu.append(end_time_gpu - start_time_gpu)

# DDraw the times in a chart
plt.figure(figsize=(10, 6))
plt.plot(sizes, times_cpu, marker='o', label='CPU')
plt.plot(sizes, times_gpu, marker='o', label='GPU')
plt.title('Execution time for matrix multiplication')
plt.xlabel('Matrix size')
plt.ylabel('Execution time (seconds)')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'sizes' is not defined